# Building a house (Harvard Business Review, 1963)

Several tasks must be completed in order to build our house. Each task has a duration, tasks may be worked on simultaneously, but there is also a precedence relation. Some tasks can only be started once other tasks have been completed. The following table shows each task, it's duration, and the tasks that must be completed before it starts. How fast can the house be built?
![alt text](https://hbr.org/resources/images/article_assets/hbr/6309/63508_A.gif)

## Problem Data

In [1]:
# this array stores the task names (:a, :b, ..., :x)
tasks = []
for i = 'a':'x'
    push!(tasks, Symbol(i))    # string(sym) converts back to a string, i.e. string(:hello) returns "hello"
end

# this dictionary stores the project durations
dur = [0, 4, 2, 4, 6, 1, 2, 3, 2, 4, 10, 3, 1, 2, 3, 2, 1, 1, 2, 3, 1, 2, 5, 0]
duration = Dict(zip(tasks,dur))

# this dictionary stores the projects that a given project depends on (ancestors)
pre = ( [], [:a], [:b], [:c], [:d], [:c], [:f], [:f], [:d], [:d,:g], [:i,:j,:h], [:k],
    [:l], [:l], [:l], [:e], [:p], [:c], [:o,:t], [:m,:n], [:t], [:q,:r], [:v], [:s,:u,:w])
pred = Dict(zip(tasks,pre));

In [4]:
# pred      # The full dictionary relating tasks to predecessors

# pred[:j]  # Predecessors to task :j


2-element Vector{Symbol}:
 :d
 :g

## Mathematical Model

Decision variables:  
Start time of the task $t_i$ for all tasks $i \in \mathcal{T}$, where $\mathcal{T}$ is the set of all tasks.
Note that we define a finish task, which defines the last "task" and has duration 0.

Constraints:  
Each task can only start when the previous task is done,  
$$t_i \geq t_j + d_j\quad \forall j \in  \mathcal{P}_i \quad \forall i \in\mathcal{T}$$
where $\mathcal{P}_i$ is the set of all tasks that must be finished before task $i$ can start (i.e., predecessor tasks) and $d_j$ is the duration of task $j$.

The first task will start at time 0, i.e.,
$$t_a == 0$$

Objective:  
$$\min_{t} t_x + d_x$$
where $t_x$ and $d_x$ are the start time and duration of the finish task, respectively. 


## JuMP Implementation

In [5]:
using JuMP,Clp
m = Model(with_optimizer(Clp.Optimizer))

# Defining the starting time as our variable
@variable(m, tstart[tasks])

# We have to start at time zero
@constraint(m, tstart[:a] == 0)

# Implementation of constraints using for-loop
for i in tasks
    for j in pred[i]
        @constraint(m, tstart[i] >= tstart[j] + duration[j])
    end
end

# # One-line implementation of the constraints:
# @constraint(m, link[i in tasks, j in pred[i]], tstart[i] >= tstart[j] + duration[j])

@objective(m, Min, tstart[:x] + duration[:x])     # total duration is start time of last task + duration of last task.

optimize!(m)
println(value.(tstart))
println("The house will take ", objective_value(m), " days to build.")
println()

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, Any[:a, :b, :c, :d, :e, :f, :g, :h, :i, :j  …  :o, :p, :q, :r, :s, :t, :u, :v, :w, :x]
And data, a 24-element Vector{Float64}:
  0.0
  0.0
  4.0
  6.0
 10.0
  6.0
  8.0
 11.0
 12.0
 10.0
 14.0
 24.0
 28.0
 27.0
 29.0
 16.0
 18.0
 18.0
 32.0
 29.0
 33.0
 19.0
 29.0
 34.0
The house will take 34.0 days to build.

Coin0506I Presolve 0 (-32) rows, 0 (-24) columns and 0 (-63) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 34
Coin0511I After Postsolve, objective 34, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 34 - 0 iterations time 0.002, Presolve 0.00


In [9]:
tstart[:x]

tstart[x]